<a href="https://colab.research.google.com/github/Bene939/BERT_News_Sentiment_Classifier/blob/main/BERT_News_Sentiment_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install pandas
!pip install pathlib
!pip install sklearn
!pip install numpy
#!pip install simpletransformers

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertTokenizer, get_linear_schedule_with_warmup, Trainer, TrainingArguments
import torch
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd
from pathlib import Path
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
from torch.nn import functional as F
from collections import defaultdict
import random

#from simpletransformers.classification import ClassificationModel

In [ ]:
#defining tokenizerm, model and optimizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)


if torch.cuda.is_available():
  print("\nUsing: ", torch.cuda.get_device_name(0))
  device = torch.device('cuda')
else:
  print("\nUsing: CPU")
  device = torch.device('cpu')
model = model.to(device)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b


Using:  Tesla T4


In [ ]:
#loading dataset
labeled_dataset = "news_headlines_sentiment.csv"
labeled_dataset_file = Path(labeled_dataset)
file_loaded = False
while not file_loaded:
  if labeled_dataset_file.exists():
    labeled_dataset = pd.read_csv(labeled_dataset_file)
    file_loaded = True
    print("Dataset Loaded")
  else:
    print("File not Found")
print(labeled_dataset)

#counting sentiments
negative = 0
neutral = 0
positive = 0
for idx, row in labeled_dataset.iterrows():
  if row["sentiment"] == 0:
    negative += 1
  elif row["sentiment"] == 1:
    neutral += 1
  else:
    positive += 1
print("Unbalanced Dataset")
print("negative: ", negative)
print("neutral: ", neutral)
print("positive: ", positive)

#balancing dataset to 1/3 per sentiment
for idx, row in labeled_dataset.iterrows():
  if row["sentiment"] == 0:
    if negative - neutral != 0:
      index_name = labeled_dataset[labeled_dataset["news"] == row["news"]].index
      labeled_dataset.drop(index_name, inplace=True)
      negative -= 1
  elif row["sentiment"] == 2:
    if positive - neutral != 0:
      index_name = labeled_dataset[labeled_dataset["news"] == row["news"]].index
      labeled_dataset.drop(index_name, inplace=True)
      positive -= 1

labeled_dataset.to_csv("test.csv")

negative = 0
neutral = 0
positive = 0
for idx, row in labeled_dataset.iterrows():
  if row["sentiment"] == 0:
    negative += 1
  elif row["sentiment"] == 1:
    neutral += 1
  else:
    positive += 1
print("Balanced Dataset:")
print("negative: ", negative)
print("neutral: ", neutral)
print("positive: ", positive)

print(labeled_dataset.reset_index(drop=True))

Dataset Loaded
                                                   news  sentiment
0     UPDATE 3-Brazil economy back to 2009 size afte...          0
1     GLOBAL MARKETS-Manufacturing data lifts stocks...          1
2     TREASURIES-Yields move higher after U.S. manuf...          2
3     UPDATE 2-Dollar weakness lifts pound to 8-mont...          2
4     UPDATE 1-U.S. House Oversight Committee to sub...          0
...                                                 ...        ...
7995  Trian Investment in Comcast Fuels Debate on Br...          0
7996                               Is Roku Stock a Buy?          1
7997                10 Most Profitable TV Shows in 2020          2
7998  Comcasts Amy Banse Transitions to Senior Advis...          1
7999  Comcast and REVOLT Sign Agreement to Expand th...          2

[8000 rows x 2 columns]
Unbalanced Dataset
negative:  2608
neutral:  1918
positive:  3474
Balanced Dataset:
negative:  1918
neutral:  1918
positive:  1918
                         

In [ ]:
#loading phrase bank dataset
phrase_bank_dataset = "all-data.csv"
phrase_bank_dataset_file = Path(phrase_bank_dataset)
file_loaded = False
while not file_loaded:
  if phrase_bank_dataset_file.exists():
    phrase_bank_dataset = pd.read_csv(phrase_bank_dataset, encoding='latin-1')
    phrase_bank_dataset = phrase_bank_dataset.values.tolist()
    file_loaded = True
    print("Dataset Loaded")
  else:
    print("File not Found")

Dataset Loaded


In [ ]:
#correcting the format of phrase bank dataset
phrase_dataset = pd.DataFrame(columns=["news", "sentiment"])
for ele in phrase_bank_dataset:
  news = ele[1]
  #converting sentiment text into numbers
  sentiment = 0 if ele[0] == 'negative' else 1 if ele[0] == 'neutral' else 2
  row = [news, sentiment]
  phrase_dataset.loc[len(phrase_dataset)] = row
print(phrase_dataset)

                                                   news sentiment
0     Technopolis plans to develop in stages an area...         1
1     The international electronic industry company ...         0
2     With the new production plant the company woul...         2
3     According to the company 's updated strategy f...         2
4     FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...         2
...                                                 ...       ...
4840  LONDON MarketWatch -- Share prices ended lower...         0
4841  Rinkuskiai 's beer sales fell by 6.5 per cent ...         1
4842  Operating profit fell to EUR 35.4 mn from EUR ...         0
4843  Net sales of the Paper segment decreased to EU...         0
4844  Sales in Finland decreased by 10.5 % in Januar...         0

[4845 rows x 2 columns]


In [ ]:
#merge both datasets
"""
final_dataset = pd.DataFrame(columns=["news", "sentiment"])
for idx,row in phrase_dataset.iterrows():
  final_dataset.loc[len(final_dataset)] = [row["news"], row["sentiment"]]
for idx,row in labeled_dataset.iterrows():
  final_dataset.loc[len(final_dataset)] = [row["news"], row["sentiment"]]
print(final_dataset)
"""

'\nfinal_dataset = pd.DataFrame(columns=["news", "sentiment"])\nfor idx,row in phrase_dataset.iterrows():\n  final_dataset.loc[len(final_dataset)] = [row["news"], row["sentiment"]]\nfor idx,row in labeled_dataset.iterrows():\n  final_dataset.loc[len(final_dataset)] = [row["news"], row["sentiment"]]\nprint(final_dataset)\n'

In [ ]:
#custom dataset class

class NewsSentimentDataset(torch.utils.data.Dataset):
  def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

  def __getitem__(self, idx):
      item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
      item['labels'] = torch.tensor(self.labels[idx])
      return item

  def __len__(self):
      return len(self.labels)

In [ ]:
#method for tokenizing dataset list

def tokenize_headlines(headlines, labels, tokenizer):
  encodings_test = tokenizer.encode_plus(
      headlines[1],
      add_special_tokens = True,
      truncation = True,
      padding = 'max_length',
      return_attention_mask = True,
      return_token_type_ids = True
  )
  encodings = tokenizer.batch_encode_plus(
      headlines,
      add_special_tokens = True,
      truncation = True,
      padding = 'max_length',
      return_attention_mask = True,
      return_token_type_ids = True
  )
  print(encodings_test)
  dataset = NewsSentimentDataset(encodings, labels)
  return dataset

In [ ]:
def count_sentiment(sentiment_list):
  #counting sentiments
  negative = 0
  neutral = 0
  positive = 0
  for ele in sentiment_list:
    if ele == 0:
      negative += 1
    elif ele == 1:
      neutral += 1
    else:
      positive += 1
  print("Dataset")
  print("negative: ", negative)
  print("neutral: ", neutral)
  print("positive: ", positive)

In [ ]:
#splitting dataset into training and validation set
#TODO: split dataset into train-val-test .7-.1-.2
#load news sentiment dataset

#all_headlines = phrase_dataset['news'].tolist()
#all_labels = phrase_dataset['sentiment'].tolist()

all_headlines = labeled_dataset['news'].tolist()
all_labels = labeled_dataset['sentiment'].tolist()

train_headlines, val_headlines, train_labels, val_labels = train_test_split(all_headlines, all_labels, test_size=.2)

count_sentiment(train_labels)
count_sentiment(val_labels)


val_dataset = tokenize_headlines(val_headlines, val_labels, tokenizer)
train_dataset = tokenize_headlines(train_headlines, val_labels, tokenizer)

Dataset
negative:  1547
neutral:  1527
positive:  1529
Dataset
negative:  371
neutral:  391
positive:  389
{'input_ids': [101, 1109, 123, 12120, 22650, 3276, 9924, 1116, 146, 112, 173, 26123, 1986, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
#data loader
train_batch_size = 8
val_batch_size = 8
#alternative to shuffle:
#sampler=RandomSampler(train_dataset)
train_data_loader = DataLoader(train_dataset, batch_size = train_batch_size, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size = val_batch_size, sampler=SequentialSampler(val_dataset))

In [ ]:
#optimizer and scheduler
num_epochs = 2
num_steps = len(train_data_loader) * num_epochs
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_steps*0.06, num_training_steps=num_steps)

In [ ]:
#simple transformers training
"""
#simple transformers model
args = {
  "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": False,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    "weight_decay": 0,
    "learning_rate": 5e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,

    "manual_seed": 64,
    "n_gpu": 1
}
model = ClassificationModel('bert', 'bert-base-cased', num_labels=3,use_cuda=True, args=args)
train,eva = train_test_split(labeled_dataset,test_size = 0.2)

train_df = pd.DataFrame({
    'text': train['news'],
    'label': train['sentiment']
})

eval_df = pd.DataFrame({
    'text': eva['news'],
    'label': eva['sentiment']
})

model.train_model(train_df)

result, model_outputs, wrong_predictions = model.eval_model(eval_df)


lst = []
for arr in model_outputs:
    lst.append(np.argmax(arr))
true = eval_df['label'].tolist()
predicted = lst
print(predicted)
print(true)
sklearn.metrics.accuracy_score(true,predicted)
"""


'\n#simple transformers model\nargs = {\n  "output_dir": "outputs/",\n    "cache_dir": "cache_dir/",\n\n    "fp16": False,\n    "fp16_opt_level": "O1",\n    "max_seq_length": 128,\n    "train_batch_size": 8,\n    "gradient_accumulation_steps": 1,\n    "eval_batch_size": 8,\n    "num_train_epochs": 1,\n    "weight_decay": 0,\n    "learning_rate": 5e-5,\n    "adam_epsilon": 1e-8,\n    "warmup_ratio": 0.06,\n    "warmup_steps": 0,\n    "max_grad_norm": 1.0,\n\n    "logging_steps": 50,\n    "save_steps": 2000,\n\n    "overwrite_output_dir": True,\n    "reprocess_input_data": False,\n\n    "manual_seed": 64,\n    "n_gpu": 1\n}\nmodel = ClassificationModel(\'bert\', \'bert-base-cased\', num_labels=3,use_cuda=True, args=args)\ntrain,eva = train_test_split(labeled_dataset,test_size = 0.2)\n\ntrain_df = pd.DataFrame({\n    \'text\': train[\'news\'],\n    \'label\': train[\'sentiment\']\n})\n\neval_df = pd.DataFrame({\n    \'text\': eva[\'news\'],\n    \'label\': eva[\'sentiment\']\n})\n\nmodel.

In [ ]:
for name, param in model.named_parameters():
	if 'classifier' not in name: # classifier layer
		param.requires_grad = False

In [ ]:
#training and evaluation
seed_val = 64

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

for epoch in range(num_epochs):

  print("\n###################################################")
  print("Epoch: {}/{}".format(epoch+1, num_epochs))
  print("###################################################\n")

  #training phase
 
  average_train_loss = 0
  average_train_acc = 0
  model.train() 
  for step, batch in enumerate(train_data_loader):
      optimizer.zero_grad()
      
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['labels'].to(device)
      token_type_ids = batch['token_type_ids'].to(device)



      outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids = token_type_ids)

      loss = F.cross_entropy(outputs[0], labels)
      average_train_loss += loss



      if step % 40 == 0:
        print("Training Loss: ", loss)


      logits = outputs[0].detach().cpu().numpy()
      label_ids = labels.to('cpu').numpy()

      print(tokenizer.batch_decode(input_ids, skip_special_tokens=True))
      print("predictions: ",np.argmax(logits, axis=1))
      print("labels:      ",label_ids)
      print("#############")

      average_train_acc += sklearn.metrics.accuracy_score(label_ids, np.argmax(logits, axis=1))
      
      
      loss.backward()
      #maximum gradient clipping
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      
      optimizer.step()
      scheduler.step()
      model.zero_grad()

  average_train_loss = average_train_loss / len(train_data_loader)
  average_train_acc = average_train_acc / len(train_data_loader)
  print("======Average Training Loss: {:.5f}======".format(average_train_loss))
  print("======Average Training Accuracy: {:.2f}%======".format(average_train_acc*100))

  #validation phase
  average_val_loss = 0
  average_val_acc = 0
  model.eval()
  for step,batch in enumerate(train_data_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)

    pred = []
    with torch.no_grad():
      

      outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

      loss = F.cross_entropy(outputs[0], labels)
      average_val_loss += loss

      logits = outputs[0].detach().cpu().numpy()
      label_ids = labels.to('cpu').numpy()
      print("predictions: ",np.argmax(logits, axis=1))
      print("labels:      ",label_ids)
      print("#############")

      average_val_acc += sklearn.metrics.accuracy_score(label_ids, np.argmax(logits, axis=1))

  average_val_loss = average_val_loss / len(val_data_loader)
  average_val_acc = average_val_acc / len(val_data_loader)

  print("======Average Validation Loss: {:.5f}======".format(average_val_loss))
  print("======Average Validation Accuracy: {:.2f}%======".format(average_val_acc*100))



###################################################
Epoch: 1/2
###################################################

Training Loss:  tensor(1.1519, device='cuda:0', grad_fn=<NllLossBackward>)
["Fed's new framework met with a confused shrug by the public - survey", 'Intel Stock Is Rising, Estée Lauder Is Falling, and the Dow Looks Set to Slide Again', 'Coca - Cola ( KO ) Gains But Lags Market : What You Should Know', 'Lockheed Martin or Raytheon : Which Defense Stock Has More Upside Potential?', 'Amazon Launches Luna, a Cloud - Based Video Game Service', 'Google - Backed American Well Stock Rises 28 % in Market Debut', 'COVID - 19 infections spike in New York as kids head back to school', 'Costco Beats on Revenue and EPS, Posts Double - Digit Q4 Comps Growth']
predictions:  [2 2 2 2 2 2 2 2]
labels:       [0 1 2 2 2 0 2 1]
#############
['Dow drops nearly 50 points on losses for shares of Goldman Sachs, UnitedHealth', 'Amgen, Honeywell Rise Premarket ; Best Buy Falls', 'The Zacks Analys

KeyboardInterrupt: ignored

In [ ]:

#test phase
average_val_loss = 0
average_val_acc = 0
model.eval()
for step,batch in enumerate(val_data_loader):
  input_ids = batch['input_ids'].to(device)
  attention_mask = batch['attention_mask'].to(device)
  labels = batch['labels'].to(device)
  token_type_ids = batch['token_type_ids'].to(device)

  pred = []
  with torch.no_grad():
    

    outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)

    loss = F.cross_entropy(outputs[0], labels)
    average_val_loss += loss

    logits = outputs[0].detach().cpu().numpy()
    label_ids = labels.to('cpu').numpy()
    print("predictions: ",np.argmax(logits, axis=1))
    print("labels:      ",label_ids)
    print("#############")

    average_val_acc += sklearn.metrics.accuracy_score(label_ids, np.argmax(logits, axis=1))

average_val_loss = average_val_loss / len(val_data_loader)
average_val_acc = average_val_acc / len(val_data_loader)

print("======Average Validation Loss: {:.5f}======".format(average_val_loss))
print("======Average Validation Accuracy: {:.2f}%======".format(average_val_acc*100))

In [ ]:
#training and evaluation with trainer moduel from huggingfaces
"""
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=0,                # number of warmup steps for learning rate scheduler
    weight_decay=0,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset  ,          # evaluation dataset
    compute_metrics=compute_metrics           
)

trainer.train()
trainer.evaluate()
"""